In [25]:
import json
import pandas as pd
from datetime import datetime
import pyodbc
import os
from dotenv import load_dotenv

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.width', 1000)

In [26]:
# Tiến hành làm sạch liệu của nfts_ìnor
with open('nguondulieu/nfts_infor.json', 'r') as file:
    nfts = json.load(file)

df_nfts = pd.DataFrame(nfts)
df_nfts

,identifier,owners_address,quantity,collection,name,update_at
0,1646,0xb2c6e86bc8b65786896a96451119bd8cac5af14e,1,pudgypenguins,Pudgy Penguin #1646,2023-07-26T15:52:34.687169
1,2022,0xd8068f813c45c88cebb02cbc0f91f95626ec9a34,1,pudgypenguins,Pudgy Penguin #2022,2024-03-28T07:40:03.185515
2,2103,0x0e91be4794fdee9b5110596021dfa60d19f0cfd5,1,pudgypenguins,Pudgy Penguin #2103,2023-07-26T16:08:50.390870
3,3211,0xc81884ceff05bd28b81ceb102106348df064285c,1,pudgypenguins,Pudgy Penguin #3211,2023-07-26T16:48:15.843914
4,3371,0x0cc3b5015c678685f3552fb1a1bfa53112b71486,1,pudgypenguins,Pudgy Penguin #3371,2023-11-30T04:01:49.150308
...,...,...,...,...,...,...
8892,14,0x076319f1d1e960f4f087c58cde74a147f51b6f57,1,pudgypenguins,Pudgy Penguin #14,2023-07-26T14:58:19.512163
8893,2,0xdc5dc5b18f09cdf1ae14862e8e4665369e614ce7,1,pudgypenguins,Pudgy Penguin #2,2024-03-07T03:44:17.418907
8894,4,0xe2e96fb14f95c9b6b892926f4ddeab6bfcfdfaff,1,pudgypenguins,Pudgy Penguin #4,2023-07-26T14:54:08.095560
8895,1,0xcce98763ff5a9ff5baf8b15abc456077a1e84f2a,1,pudgypenguins,Pudgy Penguin #1,2023-07-26T14:54:08.259610


In [27]:
# kiểm tra những giá trị null
missing_value_nft = df_nfts.isna()
for column in missing_value_nft.columns.values.tolist():
    print(missing_value_nft[column].value_counts())

identifier
False    8897
Name: count, dtype: int64
owners_address
False    8897
Name: count, dtype: int64
quantity
False    8897
Name: count, dtype: int64
collection
False    8897
Name: count, dtype: int64
name
False    8897
Name: count, dtype: int64
update_at
False    8897
Name: count, dtype: int64


In [28]:
# in ra những dòng duplacate
duplicate_indices = df_nfts.duplicated(keep=False)
duplicate_rows = df_nfts[duplicate_indices]
print(duplicate_rows)

     identifier                              owners_address  quantity     collection                 name                   update_at
32         7280  0xa679239fffd1bdad2dedce851595d21d89837161         1  pudgypenguins  Pudgy Penguin #7280  2023-07-26T19:14:55.809911
35         7454  0x51daa446b3934bbacb17db1857ee17b1d9c60dd6         1  pudgypenguins  Pudgy Penguin #7454  2023-12-16T17:53:01.897801
1041       7902  0x81c3e8b7cb0c8fde07d807138116982a9e581c39         1  pudgypenguins  Pudgy Penguin #7902  2023-07-26T19:37:01.326226
1042       7902  0x81c3e8b7cb0c8fde07d807138116982a9e581c39         1  pudgypenguins  Pudgy Penguin #7902  2023-07-26T19:37:01.326226
1584       7454  0x51daa446b3934bbacb17db1857ee17b1d9c60dd6         1  pudgypenguins  Pudgy Penguin #7454  2023-12-16T17:53:01.897801
1681       7280  0xa679239fffd1bdad2dedce851595d21d89837161         1  pudgypenguins  Pudgy Penguin #7280  2023-07-26T19:14:55.809911
2031       6899  0x0a8f4037729accb854da856431a87b8a264d8c40   

In [29]:
# xóa những dòng duplicate những token này chỉ có 1
df_nfts.drop_duplicates(subset=['identifier'], keep='first', inplace=True)
print(len(df_nfts))

8887


In [30]:
# chuyển update_at thành kiểu dữ liệu datetime
df_nfts['update_at'] = pd.to_datetime(df_nfts['update_at'])

# chia update_at thành date, time (hh:mm:ss không lấy số thập phân phía sau)
df_nfts['date'] = df_nfts['update_at'].dt.date
df_nfts['time'] = df_nfts['update_at'].dt.strftime('%H:%M:%S')

# Display the dataframe
df_nfts

,identifier,owners_address,quantity,collection,name,update_at,date,time
0,1646,0xb2c6e86bc8b65786896a96451119bd8cac5af14e,1,pudgypenguins,Pudgy Penguin #1646,2023-07-26 15:52:34.687169,2023-07-26,15:52:34
1,2022,0xd8068f813c45c88cebb02cbc0f91f95626ec9a34,1,pudgypenguins,Pudgy Penguin #2022,2024-03-28 07:40:03.185515,2024-03-28,07:40:03
2,2103,0x0e91be4794fdee9b5110596021dfa60d19f0cfd5,1,pudgypenguins,Pudgy Penguin #2103,2023-07-26 16:08:50.390870,2023-07-26,16:08:50
3,3211,0xc81884ceff05bd28b81ceb102106348df064285c,1,pudgypenguins,Pudgy Penguin #3211,2023-07-26 16:48:15.843914,2023-07-26,16:48:15
4,3371,0x0cc3b5015c678685f3552fb1a1bfa53112b71486,1,pudgypenguins,Pudgy Penguin #3371,2023-11-30 04:01:49.150308,2023-11-30,04:01:49
...,...,...,...,...,...,...,...,...
8892,14,0x076319f1d1e960f4f087c58cde74a147f51b6f57,1,pudgypenguins,Pudgy Penguin #14,2023-07-26 14:58:19.512163,2023-07-26,14:58:19
8893,2,0xdc5dc5b18f09cdf1ae14862e8e4665369e614ce7,1,pudgypenguins,Pudgy Penguin #2,2024-03-07 03:44:17.418907,2024-03-07,03:44:17
8894,4,0xe2e96fb14f95c9b6b892926f4ddeab6bfcfdfaff,1,pudgypenguins,Pudgy Penguin #4,2023-07-26 14:54:08.095560,2023-07-26,14:54:08
8895,1,0xcce98763ff5a9ff5baf8b15abc456077a1e84f2a,1,pudgypenguins,Pudgy Penguin #1,2023-07-26 14:54:08.259610,2023-07-26,14:54:08


In [31]:
# Xóa những cột không cần thiết
# xóa update_at
df_nfts = df_nfts.drop(columns=['update_at'])
df_nfts

,identifier,owners_address,quantity,collection,name,date,time
0,1646,0xb2c6e86bc8b65786896a96451119bd8cac5af14e,1,pudgypenguins,Pudgy Penguin #1646,2023-07-26,15:52:34
1,2022,0xd8068f813c45c88cebb02cbc0f91f95626ec9a34,1,pudgypenguins,Pudgy Penguin #2022,2024-03-28,07:40:03
2,2103,0x0e91be4794fdee9b5110596021dfa60d19f0cfd5,1,pudgypenguins,Pudgy Penguin #2103,2023-07-26,16:08:50
3,3211,0xc81884ceff05bd28b81ceb102106348df064285c,1,pudgypenguins,Pudgy Penguin #3211,2023-07-26,16:48:15
4,3371,0x0cc3b5015c678685f3552fb1a1bfa53112b71486,1,pudgypenguins,Pudgy Penguin #3371,2023-11-30,04:01:49
...,...,...,...,...,...,...,...
8892,14,0x076319f1d1e960f4f087c58cde74a147f51b6f57,1,pudgypenguins,Pudgy Penguin #14,2023-07-26,14:58:19
8893,2,0xdc5dc5b18f09cdf1ae14862e8e4665369e614ce7,1,pudgypenguins,Pudgy Penguin #2,2024-03-07,03:44:17
8894,4,0xe2e96fb14f95c9b6b892926f4ddeab6bfcfdfaff,1,pudgypenguins,Pudgy Penguin #4,2023-07-26,14:54:08
8895,1,0xcce98763ff5a9ff5baf8b15abc456077a1e84f2a,1,pudgypenguins,Pudgy Penguin #1,2023-07-26,14:54:08


In [32]:
# thông tin kết nối
load_dotenv()
server = os.getenv('SERVER')
database = os.getenv('DATABASE')

if server is None:
    print("Environment variable 'SERVER' is not set.")
if database is None:
    print("Environment variable 'DATABASE' is not set.")
try:
    # Ctạo kết nối
    connection = pyodbc.connect(
        'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + 
        ';DATABASE=' + database + 
        ';Trusted_Connection=yes;'
    )

    cursor = connection.cursor()

    # định nghĩa tên bảng
    collection_table_name = 'collections'
    table_name = 'nfts_infor'

    # kiểm tra bảng tồn tại hay chưa
    create_table_query = f'''
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='{table_name}' AND xtype='U')
    CREATE TABLE {table_name} (
        tokenid INT,
        name VARCHAR(50) PRIMARY KEY,
        owners_address VARCHAR(50),
        collection VARCHAR(50),
        quantity INT,
        update_date DATE,
        update_time TIME,
        FOREIGN KEY (collection) REFERENCES {collection_table_name}(collection_slug),
    )
    '''

    cursor.execute(create_table_query)
    connection.commit()

    # câu lệnh insert
    merge_query = f'''
    MERGE INTO {table_name} AS target
    USING (VALUES (?, ?, ?, ?, ?, ?, ?)) AS source (tokenid, name, owners_address, collection, quantity, update_date, update_time)
    ON target.name = source.name
    WHEN MATCHED THEN 
        UPDATE SET tokenid = source.tokenid, owners_address = source.owners_address, collection = source.collection, quantity = source.quantity, update_date = source.update_date, update_time = source.update_time
    WHEN NOT MATCHED THEN
        INSERT (tokenid, name, owners_address, quantity, collection, update_date, update_time)
        VALUES (source.tokenid, source.name, source.owners_address, source.quantity, source.collection, source.update_date, source.update_time);
    '''

    # Iinsert dữ liệu
    for index, row in df_nfts.iterrows():
        cursor.execute(merge_query, row['identifier'], row['name'], row['owners_address'], row['collection'], row['quantity'], row['date'], row['time'])
        connection.commit()

    # đóng kết nối
    cursor.close()
    connection.close()

except pyodbc.Error as ex:
    print("An error occurred while connecting to the database:", ex)